# ❄️ Snowflake Cortex: Simple RAG Example ❄️

In [ ]:
#Import python packages & establish session
import pandas as pd
from PyPDF2 import PdfFileReader
from snowflake.snowpark.files import SnowflakeFile
from io import BytesIO
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter

from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
ls @hol.public.pdf

In [ ]:
# Create a Snowpark based function to extract text from PDFs
def readpdf(file_path):
    whole_text = ""
    with SnowflakeFile.open(file_path, 'rb') as file:
        f = BytesIO(file.readall())
        pdf_reader = PdfFileReader(f)
        whole_text = ""
        for page in pdf_reader.pages:
            whole_text += page.extract_text()
    return whole_text

In [ ]:
# Register the UDF. 
# Convert the cell to markdown to prevent rerunning later.
session.udf.register(
    func = readpdf
  , return_type = StringType()
  , input_types = [StringType()]
  , is_permanent = True
  , name = 'SNOWPARK_PDF'
  , replace = True
  , packages=['snowflake-snowpark-python','pypdf2']
  , stage_location = 'hol.public.PDF'
)

In [ ]:
CREATE OR REPLACE TABLE RAW_TEXT AS
SELECT 
    relative_path, 
    file_url, 
    snowpark_pdf(build_scoped_file_url(@hol.public.PDF, relative_path)) as raw_text
from directory(@hol.public.PDF)
WHERE RELATIVE_PATH LIKE '%.pdf';

In [ ]:
select * from raw_text limit 10;

## A note on chunking
Chunking is the process of splitting a large body of text into smaller 'chunks' whilst attempting to keep as much relevant information as possible. Make the chunks too small and you run the risk of removing key information that the model requires to answer the question. Too large and it may be harder to retreive the correct body of text from the vector search - or spend tokens excessively.

There are many strategies towards chunking. Eg - pass the most relevant, top n relevant chunks, or pass the most relevent chunk + the chunk either side of that one. Play around and see what works for your use case!

In [ ]:

#A class for chunking text and returning a table via UDTF
class text_chunker:

    def process(self,text):        
        text_raw=[]
        text_raw.append(text) 
        
        text_splitter = RecursiveCharacterTextSplitter(
            separators = ["\n"], # Define an appropriate separator. New line is good typically!
            chunk_size = 1000, #Adjust this as you see fit
            chunk_overlap  = 50, #This let's text have some form of overlap. Useful for keeping chunks contextual
            length_function = len,
            add_start_index = True #Optional but useful if you'd like to feed the chunk before/after
        )
    
        chunks = text_splitter.create_documents(text_raw)
        df = pd.DataFrame(chunks, columns=['chunks','meta'])
        
        yield from df.itertuples(index=False, name=None)

In [ ]:
#Register the UDTF - set the stage location
schema = StructType([
     StructField("chunk", StringType()),
     StructField("meta", StringType()),
 ])

session.udtf.register( 
    handler = text_chunker,
    output_schema= schema, 
    input_types = [StringType()] , 
    is_permanent = True , 
    name = 'CHUNK_TEXT' , 
    replace = True , 
    packages=['pandas','langchain'], stage_location = 'HOL.PUBLIC.PDF' )

In [ ]:
--Create the chunked version of the table
CREATE OR REPLACE TABLE CHUNK_TEXT AS
SELECT
        relative_path,
        func.*
    FROM raw_text AS raw,
         TABLE(chunk_text(raw_text)) as func;

In [ ]:
select * from chunk_text limit 10;

In [ ]:
--Convert your chunks to embeddings
CREATE OR REPLACE TABLE VECTOR_STORE AS
SELECT
RELATIVE_PATH as EPISODE_NAME,
CHUNK AS CHUNK,
snowflake.cortex.embed_text('e5-base-v2', chunk) as chunk_embedding
FROM CHUNK_TEXT;

In [ ]:
--Vector distance allows use to find the most similar chunk to a question
SELECT EPISODE_NAME, CHUNK from VECTOR_STORE
            ORDER BY VECTOR_L2_DISTANCE(
            snowflake.cortex.embed_text('e5-base-v2', 
            'What is Snowpark?'
            ), CHUNK_EMBEDDING
            ) limit 1
        ;

In [ ]:
--Pass the chunk we need along with the prompt to get a better structured answer from the LLM
SELECT snowflake.cortex.complete(
    'mistral-7b', 
    CONCAT( 
        'Answer the question based on the context. Be concise.','Context: ',
        (
            SELECT chunk FROM VECTOR_STORE 
            ORDER BY vector_l2_distance(
            snowflake.cortex.embed_text('e5-base-v2', 
            'What is Snowflake Cortex?'
            ), chunk_embedding
            ) LIMIT 1
        ),
        'Question: ', 
        'What is Snowflake Cortex?',
        'Answer: '
    )
) as response;